In [67]:
using Distributions
using ReactiveMP
using Rx
using BenchmarkTools

import Base: show

In [60]:
mutable struct InferenceActor <: Actor{AbstractMessage}
    index    :: Int
    size     :: Int
    data     :: Vector{Float64}
    means    :: Vector{Float64}
    complete :: Channel{Bool}
    
    y          :: ObservedVariable
    e_mean     :: EstimatedVariable
    e_variance :: EstimatedVariable
    
    InferenceActor(data::Vector{Float64}, y::ObservedVariable, e_mean::EstimatedVariable, e_variance::EstimatedVariable) = begin
        size  = length(data)
        means = Vector{Float64}(undef, size)
        return new(1, size, data, means, Channel{Bool}(0), y, e_mean, e_variance)
    end
end

function update!(actor::InferenceActor, mean::Float64, variance::Float64)
    next!(actor.y.values, actor.data[actor.index])
    next!(actor.e_mean.values, mean)
    next!(actor.e_variance.values, variance)
end

function stop!(actor::InferenceActor)
    complete!(actor.y.values)
    complete!(actor.e_mean.values)
    complete!(actor.e_variance.values)
end

function Rx.on_next!(actor::InferenceActor, data::AbstractMessage)
    m = mean(data.distribution)
    v = var(data.distribution)
    
    actor.means[actor.index] = m
    
    actor.index += 1
    
    @async begin
            if actor.index < actor.size + 1
                update!(actor, m, v)
            else
                stop!(actor)
            end
    end
end

Rx.on_error!(actor::InferenceActor, err) = error(err)
Rx.on_complete!(actor::InferenceActor)   = put!(actor.complete, true)

Base.show(io::IO, actor::InferenceActor) = print(io, "InferenceActor")

In [69]:
@btime begin
    N = 20000
    data = collect(1:N) + rand(Normal(0.0, 1.0), N);
    
    x_prev_add   = addition_node("x_prev_add", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
    add_1        = constant_variable("add_1", 1.0, x_prev_add.in2);

    x_prev_prior = gaussian_mean_variance("x_prev_prior");
    x_prev_m     = estimated_variable("x_prev_m", x_prev_prior.mean);
    x_prev_v     = estimated_variable("x_prev_v", x_prev_prior.variance);

    x_prev = random_variable("x_prev", x_prev_prior.value, x_prev_add.in1);

    noise_node     = gaussian_mean_variance("noise_node");
    noise_mean     = constant_variable("noise_mean", 0.0, noise_node.mean);
    noise_variance = constant_variable("noise_variance", 1.0, noise_node.variance);

    add_x_and_noise = addition_node("add_x_and_noise", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);

    x = random_variable("x", x_prev_add.out, add_x_and_noise.in1);
    n = random_variable("n", noise_node.value, add_x_and_noise.in2);
    y = observed_variable("y", add_x_and_noise.out);
    
    actor = InferenceActor(data, y, x_prev_m, x_prev_v);
    
    @async begin
        subscribe!(inference(x), actor)   
        update!(actor, 0.0, 1000.0) 
    end
    
    yield()
    
    take!(actor.complete)
    
    return actor.means
end;

  2.447 s (1100488 allocations: 46.41 MiB)


In [66]:
@time actor = try_reactive_1()

  1.300438 seconds (550.47 k allocations: 23.217 MiB)


InferenceActor